In [2]:
import pandas as pd
import openpyxl
# packages

# Load and Calculate Inflation Data

In [108]:
import pandas as pd

def process_exchange_rate_data(file_path, sheet_name, exchange_rate_column, new_column_name):
    # Read the Excel file with the specified header row
    exchange_rate_df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Select the relevant columns and sort by 'Year'
    exchange_rate_df = exchange_rate_df[['Year', exchange_rate_column]]
    exchange_rate_df = exchange_rate_df.sort_values(by='Year', ascending=True)

    # Calculate the change in exchange rate and add it as a new column
    exchange_rate_df[new_column_name] = exchange_rate_df[exchange_rate_column].diff()

    return exchange_rate_df

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'data/exchangerates.xlsx'

chf_euro = process_exchange_rate_data(file_path, 'CHF_EUR', 'Average CHF/EUR', 'Exchange Rate Change') # Process CHF/EUR data
chf_usd = process_exchange_rate_data(file_path, 'CHF_USD', 'Average CHF/USD', 'Exchange Rate Change') # Process CHF/USD data


In [109]:
# Specify the dynamic year you want to use as the baseline
dynamic_year = 1983  # Change this to the desired year min 1983 max 2022

In [119]:

import pandas as pd

def process_inflation_data(file_path, sheet_name, last_valid_entry, dynamic_year, exchange_rate_df):
    # Read the Excel file with the specified header row
    data = pd.read_excel(file_path, sheet_name=sheet_name, header=3)

    # Set the dynamic year values to 100 and remove columns before that year
    df = data.iloc[:last_valid_entry + 1]  # This will keep rows up to the last_valid_entry

    df_2 = df[df["Level"] == 2]

    # Select the columns you want to adjust in df_2
    start_year = dynamic_year + 1
    end_year = 2022

    # Generate a list of integers from start_year to end_year
    cols_to_adjust = list(range(start_year, end_year + 1))

    if exchange_rate_df is not None:

        # Create a dictionary from exchange rate DataFrame
        exchange_rate_change_dict = exchange_rate_df.set_index('Year')['Exchange Rate Change'].to_dict()

        # Adjust the values in df_2 based on the exchange rate change
        for col in cols_to_adjust:
            year = int(col)
            df_2[col] = df_2[col] + (exchange_rate_change_dict.get(year, 0) * 100)

    # Set the dynamic year values to 100
    df_2.loc[:, dynamic_year] = 100

    # Loop through years and calculate real values and subtract 100, starting from the dynamic year
    for year in range(dynamic_year + 1, 2023):
        df_2[year] = df_2[year - 1] * (1 + df_2[year] / 100)

    for year in range(dynamic_year, 2023):
        df_2[year] = df_2[year] - 100

    subset_columns = ['PosTxt_E', 'PosNo', dynamic_year] + list(range(dynamic_year + 1, 2023))
    df_inflation = df_2[subset_columns]
    df_inflation = df_inflation.rename(columns={'PosNo': 'ID'})

    return df_inflation

# Example usage with chf_euro DataFrame
file_path = 'data/su-e-05.02.67.xlsx'
sheet_name = 'VAR_y-1'
last_valid_entry = 415
dynamic_year = 1983

# Replace 'chf_euro' with the desired exchange rate DataFrame
exchange_rate_df = chf_usd

# Process inflation data with the specified exchange rate DataFrame
df_inflation = process_inflation_data(file_path, sheet_name, last_valid_entry, dynamic_year, exchange_rate_df)


/var/folders/0q/zp8d2_c16z32wsh2hdjb9fch0000gn/T/ipykernel_37365/2779947462.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2[col] = df_2[col] + (exchange_rate_change_dict.get(year, 0) * 100)
/var/folders/0q/zp8d2_c16z32wsh2hdjb9fch0000gn/T/ipykernel_37365/2779947462.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2[year] = df_2[year - 1] * (1 + df_2[year] / 100)
/var/folders/0q/zp8d2_c16z32wsh2hdjb9fch0000gn/T/ipykernel_37365/2779947462.py:39: SettingWithCopyWarning: 
A value is trying to b

In [120]:
df_inflation

,PosTxt_E,ID,1983,1984,1985,1986,1987,1988,1989,1990,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,Food and non-alcoholic beverages,1.0,0.0,-1.1385,-0.104804,15.474751,29.416595,33.951482,25.606572,46.456635,...,125.406852,130.74133,116.48267,112.056249,113.028526,117.187908,113.931393,127.046029,129.591215,122.892203
100,Alcoholic beverages and tobacco,2.0,0.0,-4.8385,-4.41446,11.735577,25.337707,27.348249,21.324932,40.736314,...,198.34058,205.699449,189.254347,180.736674,182.304589,186.122758,183.835493,203.78998,212.967475,203.209775
117,Clothing and footwear,3.0,0.0,-1.9385,0.361631,18.623332,34.487424,38.527561,31.421374,51.133923,...,109.999774,110.349634,99.664924,97.37936,103.021449,107.594507,108.218536,121.399394,126.538074,120.60776
153,Housing and energy,4.0,0.0,-2.1385,-0.234507,12.631152,26.905461,30.717828,27.149492,51.943008,...,233.351552,241.574001,221.149584,212.97729,216.91611,223.103897,219.228589,237.840573,252.100824,250.629746
191,Household goods and services,5.0,0.0,-3.8385,-3.313843,13.11884,27.454956,30.391646,23.572423,42.231241,...,113.969376,114.967757,98.889244,89.651435,86.348655,86.632837,86.820589,97.339149,105.471495,106.051336
231,Healthcare,6.0,0.0,-1.7385,0.664583,17.773435,34.230506,40.813304,33.167424,53.408206,...,166.837384,168.349284,152.840037,145.647497,144.562963,143.713107,139.327733,151.845292,157.690621,144.760221
249,Transport,7.0,0.0,-3.4385,-0.883869,11.501584,25.186731,28.947465,26.974827,47.290164,...,173.698163,174.427842,147.590171,135.59517,139.266921,147.288345,144.322368,150.74878,167.601605,179.060841
284,Communications,8.0,0.0,9.6615,10.698459,26.191705,34.7393,29.220513,19.232026,32.466184,...,0.357254,-0.479123,-6.728537,-10.407816,-11.78888,-10.860458,-11.573039,-6.417482,-4.619762,-9.596512
291,Recreation and culture,9.0,0.0,-2.5385,-0.739751,16.924504,31.742931,35.700621,29.282253,48.027533,...,109.586447,112.869828,97.161099,93.91839,95.971016,100.581235,96.771395,103.324472,107.230335,101.598229
353,Education,10.0,0.0,0.4615,2.817925,24.302656,44.531048,52.630713,47.853677,70.622404,...,309.174726,321.722478,304.096587,297.450411,301.259973,308.292054,304.619875,332.256626,347.044126,328.635743


In [107]:
import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'data/su-e-05.02.67.xlsx'

# Replace 'INDEX_y' with the name of the sheet you want to read
sheet_name = 'VAR_y-1'

# Read the Excel file with the specified header row
data = pd.read_excel(file_path, sheet_name=sheet_name, header=3)

# Last valid entry in excel
last_valid_entry = 415  # You can adjust this value based on your data

# Set the dynamic year values to 100 and remove columns before that year
df = data.iloc[:last_valid_entry + 1]  # This will keep rows up to the last_valid_entry

df_2 = df[df["Level"] == 2]

# Create a dictionary from chf_euro DataFrame
exchange_rate_change_dict = chf_euro.set_index('Year')['Exchange Rate Change'].to_dict()
# Select the columns you want to adjust in df_2
start_year = dynamic_year+1
end_year = 2022

# Generate a list of integers from start_year to end_year
cols_to_adjust = list(range(start_year, end_year + 1))

# Adjust the values in df_2 based on the exchange rate change
for col in cols_to_adjust:
    year = int(col)
    df_2[col] = df_2[col] + (exchange_rate_change_dict.get(year, 0)*100)

# Set the dynamic year values to 100
df_2.loc[:, dynamic_year] = 100

# Loop through years and calculate real values and subtract 100, starting from the dynamic year
for year in range(dynamic_year + 1, 2023):
    df_2[year] = df_2[year - 1] * (1 + df_2[year] / 100)

for year in range(dynamic_year, 2023):
    df_2[year] = df_2[year] - 100

subset_columns = ['PosTxt_E', 'PosNo', dynamic_year] + list(range(dynamic_year + 1, 2023))
df_inflation = df_2[subset_columns]
df_inflation = df_inflation.rename(columns={'PosNo': 'ID'})

/var/folders/0q/zp8d2_c16z32wsh2hdjb9fch0000gn/T/ipykernel_37365/2132209679.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2[col] = df_2[col] + (exchange_rate_change_dict.get(year, 0)*100)
/var/folders/0q/zp8d2_c16z32wsh2hdjb9fch0000gn/T/ipykernel_37365/2132209679.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2[year] = df_2[year - 1] * (1 + df_2[year] / 100)
/var/folders/0q/zp8d2_c16z32wsh2hdjb9fch0000gn/T/ipykernel_37365/2132209679.py:42: SettingWithCopyWarning: 
A value is trying to be 

Ajust to Euro 

In [96]:
csv_file_path = "data/df_inflation.csv"  # Replace "your_file_path_here.csv" with the desired file path
df_inflation.to_csv(csv_file_path, index=False)  # Setting index=False to exclude the index column from the CSV

In [95]:
df_inflation

,PosTxt_E,ID,1983,1984,1985,1986,1987,1988,1989,1990,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,Food and non-alcoholic beverages,1.0,0.0,4.4147,6.961166,9.74836,11.725806,14.241536,12.63907,19.713479,...,71.55732,74.987266,93.539766,90.446033,88.012323,83.968178,90.237445,97.092652,92.174205,109.070737
100,Alcoholic beverages and tobacco,2.0,0.0,0.7147,2.566642,6.57269,8.599489,8.981434,9.087473,15.284296,...,127.4568,132.2318,158.711335,152.247432,149.27621,142.418621,152.376693,163.742478,162.171891,184.698487
117,Clothing and footwear,3.0,0.0,3.6147,7.488646,13.08429,16.591939,18.63428,18.275174,23.811162,...,60.553611,60.231379,78.981976,77.731787,79.726115,76.399385,85.585912,92.644672,90.14896,107.437874
153,Housing and energy,4.0,0.0,3.4147,6.86751,6.766841,9.331167,11.246321,13.57949,24.006769,...,152.936578,158.246476,186.142777,180.13807,178.799291,172.802318,182.917282,192.263455,194.030187,227.232958
191,Household goods and services,5.0,0.0,1.7147,3.686745,7.840228,10.430335,11.591731,11.142351,16.455622,...,63.011815,63.173685,78.352428,70.864301,64.921812,58.405751,66.338394,71.500706,72.365927,93.036566
231,Healthcare,6.0,0.0,3.8147,7.799939,12.118189,16.26858,20.514359,19.788019,25.634394,...,103.038116,103.442771,125.825952,120.409516,115.609217,106.012451,112.414906,118.157543,115.331312,129.74193
249,Transport,7.0,0.0,2.1147,6.238909,5.713876,7.830056,9.718915,13.336677,19.77488,...,107.259255,107.050545,121.548639,111.803822,111.427023,109.839206,117.619624,117.84486,124.607854,160.527816
284,Communications,8.0,0.0,15.2147,17.909341,19.095274,14.929083,8.782331,5.624706,6.449212,...,-22.894993,-23.820792,-15.8203,-18.765327,-21.428118,-23.825346,-20.696244,-18.076444,-19.465458,-14.236914
291,Recreation and culture,9.0,0.0,3.0147,6.35114,11.462163,14.139261,16.138639,16.251642,21.111658,...,59.217885,61.127385,76.276904,74.164225,72.983566,69.954624,75.066519,76.473178,73.481076,89.081014
353,Education,10.0,0.0,6.0147,9.978378,18.673048,25.795567,31.269813,33.497855,40.280347,...,212.084746,220.508849,260.900016,256.574629,253.800835,245.482977,259.329244,274.792978,274.060258,302.460035


# Load inflation basket composition

In [7]:
valid_gp_values = [1,2,3,4,5,6,7,8,9,10,11,12]

def process_excel_data(file_path,
                       sheet_name,
                       header_row,
                       last_valid_entry,
                       valid_column,
                       year_columns,
                       id_column,
                       id_column_name,
                       additional_columns=None):
    
    data = pd.read_excel(file_path, sheet_name=sheet_name, header=header_row)
    data = data.iloc[:last_valid_entry + 1]

    filtered_data = data[data[valid_column].isin(valid_gp_values)]
    selected_columns = additional_columns + [valid_column] + year_columns
    filtered_data = filtered_data[selected_columns]
    filtered_data = filtered_data.rename(columns={valid_column: id_column_name})

    return filtered_data

file_path = 'data/su-d-05.02.90.xlsx'

# Process 2020 data
df_b_l2_20 = process_excel_data(file_path, 'LIK2020', 3, 415, 'PosNo', [2021, '2022', '2023'], 'PosNo', 'ID', additional_columns=['PosTxt_E'])

# Process 2015 data
valid_PosNo_15 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df_b_l2_15 = process_excel_data(file_path, 'LIK2015', 3, 396, 'PosNo', [2016, 2017, '2018', '2019', '2020'], 'PosNo', 'ID', additional_columns=['PosTxt_E'])

# Process 2010 data
valid_gp_values_10 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df_b_l2_10 = process_excel_data(file_path, 'LIK2010', 3, 313, 'GP Nr.', [2011, 2012, 2013, 2014, 2015], 'GP Nr.', 'ID', additional_columns=['deutsch'])

# Process 2005 data
valid_gp_values_05 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df_b_l2_05 = process_excel_data(file_path, 'LIK2005', 3, 313, 'GP Nr.', [2006, 2007, 2008, 2009, 2010], 'GP Nr.', 'ID', additional_columns=['deutsch'])

# Process 2000 data
valid_gp_values_00 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df_b_l2_00 = process_excel_data(file_path, 'LIK2000', 3, 313, 'Nr. ', ['2000/01', 2002, 2003, 2004, 2005], 'Nr. ', 'ID', additional_columns=['Position'])

# Start with df_b_l2_00 and perform left joins in the reverse order
left_merged_df = pd.merge(df_b_l2_00, df_b_l2_05, on='ID', how='left')
left_merged_df = pd.merge(left_merged_df, df_b_l2_10, on='ID', how='left')
left_merged_df = pd.merge(left_merged_df, df_b_l2_15, on='ID', how='left')
left_merged_df = pd.merge(left_merged_df, df_b_l2_20, on='ID', how='left')
left_merged_df.columns = left_merged_df.columns.astype(str)
df_weights = left_merged_df[['Position','ID','2000/01','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']]
df_weights = df_weights.rename(columns={'2000/01': '2001'})
columns_to_convert = df_weights .columns[2:]  # Assuming the percentage columns start from the 3rd column
df_weights[columns_to_convert] = df_weights [columns_to_convert] / 100 # convert to decimal


In [8]:


csv_file_path = "data/df_weights.csv"  # Replace "your_file_path_here.csv" with the desired file path
df_weights.to_csv(csv_file_path, index=False)  # Setting index=False to exclude the index column from the CSV